In [43]:
import google.generativeai as genai
import os
import json
from openai import OpenAI
import tqdm
import hashlib
import time
import threading

In [6]:
openai_key = open(os.path.expanduser('~/.openai_key.txt')).read().strip()
client = OpenAI(api_key=openai_key)

In [7]:
stories = json.load(open('spanish_stories/stories_00.json', 'r'))

In [8]:
print(stories[0]['story'])

Un día, un niño llamado Leo jugó con cartas. Él tomó las cartas y las puso en un montón. Después, él las puso en otro montón. ¡Él inventó un nuevo juego! 

Leo miró las cartas. Él vio un gato. "¡Miau!" dijo el gato. Después, él vio un perro. "¡Guau!" dijo el perro. Leo sonrió. Él pensó que su juego era divertido. 

Después de jugar, Leo se fue a la cama. Él soñaba con el gato y el perro. En su sueño, el gato y el perro corrieron juntos. Ellos eran felices.

¡De repente! Leo despertó. Él sintió que algo no estaba bien. Él fue a la ventana. Él vio al gato y al perro jugando afuera. ¡El sueño se había hecho realidad! 

Leo corrió afuera y jugó con el gato y el perro. Él jugó despacio. Él no quería asustarlos. Él era feliz. Él sabía que su sueño había sido real. 



In [13]:
def make_translation_request(story):
    prompt = f"""You are tasked with translating a children's story from Spanish to English. 
Your goal is to create an accurate translation that maintains the original story's tone, style, and 
meaning while making it accessible to English-speaking children who only know simple words. 
Here is the Spanish story <spanish_story>{story}</spanish_story> 
Present your translation within <english_translation> tags."""
    return prompt
    

In [14]:
print(make_translation_request(stories[0]['story']))

You are tasked with translating a children's story from Spanish to English. 
Your goal is to create an accurate translation that maintains the original story's tone, style, and 
meaning while making it accessible to English-speaking children who only know simple words. 
Here is the Spanish story <spanish_story>Un día, un niño llamado Leo jugó con cartas. Él tomó las cartas y las puso en un montón. Después, él las puso en otro montón. ¡Él inventó un nuevo juego! 

Leo miró las cartas. Él vio un gato. "¡Miau!" dijo el gato. Después, él vio un perro. "¡Guau!" dijo el perro. Leo sonrió. Él pensó que su juego era divertido. 

Después de jugar, Leo se fue a la cama. Él soñaba con el gato y el perro. En su sueño, el gato y el perro corrieron juntos. Ellos eran felices.

¡De repente! Leo despertó. Él sintió que algo no estaba bien. Él fue a la ventana. Él vio al gato y al perro jugando afuera. ¡El sueño se había hecho realidad! 

Leo corrió afuera y jugó con el gato y el perro. Él jugó despaci

In [29]:
def execute_translation_request(story_text):
    req = make_translation_request(story_text)
    return client.chat.completions.create(
        model="gpt-4o-mini",
        max_tokens=4096,
        messages = [{"role": "user", "content": req}])

def get_translation(response):
    return response.choices[0].message.content.split('<english_translation>')[1].split('</english_translation>')[0]

In [50]:
def translate_and_save(story_json, thread_id):
    resp = execute_translation_request(story_json['story'])
    translation = get_translation(resp)
    md5 = hashlib.md5(story_json['story'].encode()).hexdigest()
    story_json['translation'] = translation
    with open(f'spanish_stories_translated/story-{thread_id}-{md5}.json', 'w') as f:
        json.dump(story_json, f)

def translate_story_range(start, end, thread_id):
    successes = 0
    fails = 0
    for i in range(start, end):
        try:
            translate_and_save(stories[i], thread_id)
            successes += 1
        except Exception as e:
            print('skipping story', i, 'on thread', thread_id, 'because', e)
            time.sleep(30)
            fails += 1
    if i % 100 == 0:
        print('thread', thread_id, 'completed', i - start, 'stories', 'successes:', successes, 'fails:', fails)

In [51]:
NUM_THREADS = 8
NUM_STORIES = len(stories)

stories_per_thread = NUM_STORIES // NUM_THREADS
start_of_ranges = [i * stories_per_thread for i in range(NUM_THREADS)]
end_of_ranges = [(i + 1) * stories_per_thread for i in range(NUM_THREADS)]

threads = []

for thread_id, (start, end) in enumerate(zip(start_of_ranges, end_of_ranges)):
    t = threading.Thread(target=translate_story_range, args=(start, end, thread_id))
    threads.append(t)
    t.start()

for t in threads:
    t.join()


skipping story 62545 on thread 5 because Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-SrRUtA8yGUqebUV3XD1Jqln4 on tokens per min (TPM): Limit 60000, Used 56393, Requested 4442. Please try again in 835ms. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
skipping story 61 on thread 0 because Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-SrRUtA8yGUqebUV3XD1Jqln4 on tokens per min (TPM): Limit 60000, Used 56698, Requested 4341. Please try again in 1.039s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}
skipping story 62 on thread 0 because Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-SrRUtA8yGUqebUV3XD1Jqln4 on tokens per min (TPM): Limit 60000, Used 58637, Requested 4